In [2]:
from __future__ import division
import PyDSTool as dst
import numpy as np

#Setting up models using `PyDSTool`

This tutorial is based on the following PyDSTool documentation pages:
* http://www.ni.gsu.edu/~rclewley/PyDSTool/ModelSpec.html
* http://www.ni.gsu.edu/~rclewley/PyDSTool/Symbolic.html


Implementing a work-in-progress dynamical system in a program can become very messy, very fast, as the model evolves, while the programmer's patience with keeping all the necessary parameters, variables, and auxiliary functions organized devolves. 

The goal of `PyDSTool`'s `ModelSpec` class is to provide a class hierarchy that allows complex dynamical models to be composed from simple units of mathematical expression. The goal of this notebook is to provide documentation to help make it easy to set up complicated dynamical systems using `PyDSTool`.


#`Quantity` classes

The `Quantity` classes are the elementary model building blocks of PyDSTool --- they are the ultimate "leaves" in the hierarchy of model specification that is supported by the ModelSpec classes. The essence of symbolic manipulation is the definition of Quantity objects. There are three types of Quantity objects: `Var`, `Par`, and `Fun`.

##`Var` (Variable)

Here is a variable declaration only using a name:

In [2]:
k = dst.Var('k')

Here is a variable declaration that includes a definition:

In [3]:
v = dst.Var('-k*(1 + k)*(1/2)', 'v')

The string representation of a variable is simply its name:

In [4]:
print v

v


In [5]:
print k

k


"Calling" a variable returns its definition:

In [6]:
print v()

-k*(1+k)*(1/2)


If a `Var` (or other `Quantity`, such as `Par` or `Fun`) does not have a definition, calling it returns its name once again:

In [7]:
print k()

k


At this time there is no class method to add the specification directly to a Quantity that has only been declared. (Such a method is not really necessary when the overhead in checking the specification is almost identical to that necessary to re-create the object entirely.)

##`QuantSpec` vs. `Quantity`

`v` is a `Var` object, which is a `Quantity` object:

In [8]:
v

Var v (ExpFuncSpec)

`v()` is a `QuantSpec` object:

In [9]:
v()

QuantSpec __result__ (ExpFuncSpec)

In the documentation, one might see references to `QuantSpec` or `Quantity` separately. In this tutorial, I will mostly use "definition" and "(`Quantity` object's) name" respectively.

##`Par` (parameter)

In [10]:
p = dst.Par('p')
print p

p


In [11]:
print p()

p


In [12]:
q = dst.Par('3.5e-3', 'p')
print q

p


In [13]:
print q()

3.5e-3


##`Fun` (function)

So far, the symbols we have seen appearing in the definition are 'free', and generally refer to other Quantities. They are not treated as 'local parameters', and therefore do not behave like arguments or inputs to a function.

In [14]:
v.freeSymbols

['k']

In [15]:
f = dst.Fun('(((k-(-10)))/(3+4))', [k], 'v')

In [16]:
f.freeSymbols

[]

Note that $k$ is a free symbol for `v`, but not for `f`. $k$ is an argument of `f`, and the code `[k]` defines `f`'s "signature". We can supply a value for $k$ during evaluation. Look carefully at the following examples.

Supplying a number as an argument:

In [17]:
print f(4)

2.0


Supplying a previously defined variable:

In [18]:
print f(k)

(k+10)/7


Supplying a previously defined parameter:

In [19]:
print f(p)

(p+10)/7


Supplying another previously defined parameter -- but note that so far we have been specifically supplying names, not definitions:

In [20]:
print f(q)

(p+10)/7


We can supply the definition by calling the `Quantity`:

In [21]:
print f(q())

1.4290714285714288


In [22]:
print f(p())

(p+10)/7


Once more comparing definitions vs. names:

In [23]:
print f(v())

(-k*(1+k)*0.5+10)/7


In [24]:
print f(v)

(v+10)/7


Too many arguments:

In [25]:
f(1, 2)

ValueError: Invalid number of arguments for auxiliary function

#Expression simplification and evaluation

##Simplification

Algebraic (mostly of brackets and signs) and arithmetical simplification of the definition of a `Quantity` is done using the object's `simplify` method. `simplify` works in-place on the object, and does not return anything.

In [26]:
u = dst.Var('((-k*(-k + 2*k)*(-(-1)/(1 + 1))))', 'u')

In [27]:
print u()

((-k*(-k+2*k)*(-(-1)/(1+1))))


In [28]:
u.simplify()

In [29]:
print u()

-k*(-k+2*k)*0.5


On the other hand, the object's `eval` method returns a simplified expression for the definition, without changing the original definition.

In [30]:
w = dst.Var('((-k*(-k + 2*k)*(-(-1)/(1 + 1))))', 'w')
print w.eval()

-k*(-k+2*k)*0.5


In [31]:
print w()

((-k*(-k+2*k)*(-(-1)/(1+1))))


##Evaluation

Quantities can also be evaluated at specific values for the free symbols, but care must be taken with functions in terms of the order in which the operations will be performed.

In [32]:
print w.eval(k = 1)

-0.5


Evaluations at non-numeric values for a symbol performs symbolic substitution:

In [33]:
print w.eval(k=v)

-v*(-v+2*v)*0.5


In [34]:
print w.eval(k=v())

-(-(-k*(1+k)*0.5)*(1-k*(1+k)*0.5)*0.5)*(-(-(-k*(1+k)*0.5)*(1-k*(1+k)*0.5)*0.5)+2*-(-k*(1+k)*0.5)*(1-k*(1+k)*0.5)*0.5)*0.5


In [35]:
print w.eval(k='j')

-j*(-j+2*j)*0.5


In [36]:
print w.eval(k=q)

-p*(-p+2*p)*0.5


In [37]:
print k()

k


In [38]:
print k

k


There is another function for expression substitution, called `subs`, that can act more "intelligently" (what is meant by this?) when dealing with defined `Par` quantities. It can also take a dictionary of assignments, and the result is always as simplified as possible.

In [39]:
print dst.subs(v(), {'k': 0})

0.0


Always remember to use `subs` on the right object. For instance, if we had used it on `v`'s name:

In [40]:
print dst.subs(v, {'k': 0})

v


This call did nothing because the `Quantity` assigned to Python name `'v'` evaluates to its name, which is also `'v'`. This name does not match any of the assignments in the call to `subs`, so no substitutions are made.

Evaluation of one symbolic expression can perform automatic substitution of values defined by expressions in the local scope:

In [41]:
q = dst.Quantity('xv+1','qv')
x = dst.Quantity('3','xv')
print q.eval()

4


In [42]:
a=x/q
print a

xv/qv


In [43]:
print a.eval()

0.75


Recall how we noted the difference between `Quantity` and `QuantSpec` objects previously:

In [44]:
a_ = x()/q()
print a_

3/(xv+1)


In other words, `a_` yields the division in terms of the definitions (or, `QuantSpec`), since we specifically called for them. We can use this difference between `Quantity` and `QuantSpec` to perform partial evaluations:

In [45]:
print a.eval() # using local scope information, xv = 3

0.75


In [46]:
print a.eval(qv=q())

xv/(xv+1)


In [47]:
print a.eval(xv=5,qv=q())  # override xv=3 from local scope

0.8333333333333334


Evaluation with an explicitly-supplied scope dictionary is much faster, if the relevant definitions are known. Equivalently, these definitions can be supplied in the call as a comma-separated sequence:

In [48]:
print a.eval(xv=x())

3/qv


In [49]:
print a.eval(xv=x(),qv=q())  # much faster

0.75


In [50]:
print a.eval({'xv': x, 'qv': q()})  # equivalently fast

0.75


##Advanced information about functions

In [51]:
# some variables
a = dst.Var('a')
b = dst.Var('b')
c = dst.Par('c')

# a function
gfun = dst.Fun(a+b/c, [a, b], 'g')

In [52]:
print gfun.eval(a=4)

ValueError: All function signature arguments are necessary for evaluation

In [53]:
print gfun.eval(a=4, b=3)

4+3/c


One might wonder why `c` was not required in the `eval` call. However, if we look at `gfun`'s signature, is a function of two arguments, `'a'` and `'b'`, while `'c'` is a free variable that *must refer to a parameter name* if the `Fun` object is to be used in the specification of a right-hand side (RHS), where RHS refers to the right hand side of a system of ODEs:

\begin{align}
    \vec{x} &= \left[\begin{array}{cccc}\dot{x}_1 & \dot{x}_2 & \dots & \dot{x}_n\end{array}\right] \\
    \quad \\
    \dot{x}_1 &= f_1(\vec{x}) \\
    \dot{x}_2 &= f_2(\vec{x}) \\
    \vdots \\
    \dot{x}_n &= f_n(\vec{x})
\end{align}

In [54]:
print gfun.eval(a=4,b=3,c=3)

5.0


In [55]:
print gfun(4,b)

4+b/c


`gfun(a, b)` evaluates to the `QuantSpec` object containing the definition of the function. As this is not a function object (`Fun`) any of the free names 'a', 'b', or 'c' can be evaluated using `eval`.

In [56]:
print gfun(a, b).eval(a=4)  # the result of gfun(a, b) is not a Fun, but a QuantSpec

4+b/c


Any evaluations given for names not present in the definition of the object will be ignored.

In [57]:
print gfun.eval(a=a, b=b, d=5, c=c)

a+b/c


##More symbolic substitutions: name maps

A more efficient way to make multiple symbolic substitutions which are solely textual, i.e. do not involve algebraic simplification, is to use the `mapNames(<name_mapping_dict>)` method of a `Quantity`. The method works in-place and does not return anything, and the `Quantity` whose method was called will change.

In [58]:
v.mapNames({'k': 'a/b'})
print v()

-a/b*(1+a/b)*0.5


This method takes a dictionary of `source: target` names. Matching only occurs on whole symbols (i.e. 'tokens'): a dictionary containing the mapping (`'a': 'b'`) will not map the symbol `'ka_2'` to `'kb_2'`.

The method will also map the `Quantity`'s name only if it matches a source name:

In [59]:
v.mapNames({'a': '44', '10': '-10'})
print v()

-44/b*(1+44/b)*0.5


# Symbolic versions of built-in math names

Symbolic versions of basic math functions (`pow`, `sqrt`, `abs`, etc.) and constants are exported by `Symbolic.py`. They can be used in title-cased form (e.g. `Pow` instead of `pow`), as in Maple, but lower case form works as well. 

In [60]:
v = dst.Var('-44/b*(1+44/b)*0.5', 'v')

In [61]:
print v.eval(b = 'Pow(1, 1)')

-990.0


In [62]:
print v.eval(b = 'pow(1, 1)') # case does not matter

-990.0


In [63]:
print dst.subs(v(), {'b': 'Pow(x, y)'})

-44/Pow(x,y)*(1+44/Pow(x,y))*0.5


In [64]:
print dst.subs(v(), {'b': 'Pow(x, y)'}).eval(x = 1, y = 1)

-990.0


The functions supported included the trigonometric (`Sin`, `Cos`, `Tan`, but not inverse trigonometric functions), exponential (`Exp`) and logarithmic functions, the absolute value (`Abs`), power (`Pow`), and square root (`Sqrt`) functions. The constants include `E` and `Pi`.



In [65]:
print v.eval(b = 'Sin(Tan(Sqrt(Abs(Pow((Cos(Pi*E)), 1)))))')

-1357.3899529842106


In [66]:
z = dst.Var('Log(E)', 'z')
print z()

Log(E)


In [67]:
print z.eval()

1.0


In [68]:
z = dst.Var('Arccos(1)', 'z')
print z.eval()

Arccos(1)


#Multi-reference quantities

There is a way to specify a range of related Quantities at once, in a kind of macro. This is useful for repeated definitions, that perhaps are interconnected in a formulaic way. (You may be familiar with this kind of notation in the package XPP.)

In [69]:
v = dst.Var('v')
ipar = dst.Par('ipar')

z = dst.Var('3+v/((1+i)*ipar)', 'z[i,0,5]') # note z[i,0,5]

These define six `uantities`, `z0` to `z5`, that are referenced from the Python object `z` as `z[0]` to `z[5]`. 

In [70]:
print z[0]().eval()

3+1.71428571429/ipar


In [71]:
print z[1]().eval()

3+1.71428571429/(2*ipar)


The following example shows how to define a set of ODE right-hand sides that involve a circular connectivity pattern. Notice that the boundary variables `w0` and `w3` have to be defined separately, so that the multi-quantity definition refers to valid `Quantity` objects for all index values.

In [72]:
w0 = dst.Var('w0-2*(w3-w1)', 'w0', specType='RHSfuncSpec') # boundary value
w3 = dst.Var('w3-2*(w2-w0)', 'w3', specType='RHSfuncSpec') # boundary value
wi = dst.Var('w[i]-2*(w[i-1]-w[i+1])', 'w[i,1,2]', specType='RHSfuncSpec')

Alternatively, we could have used the modulo function (`%`) to give a cleaner definition:

In [73]:
for i in (np.arange(6) - 1):
    print "{}%3: {}".format(i, i%3)

-1%3: 2
0%3: 0
1%3: 1
2%3: 2
3%3: 0
4%3: 1


In [74]:
wi = dst.Var('w[i]-2*(w[(i-1)%3]-w[(i+1)%3])', 'w[i,0,3]', specType='RHSfuncSpec')

Note that the Python dereferencing of a multi-def `z` by (for example) `z[2]` actually creates an *instance* of `z2`. (What does this mean?)

In [75]:
print z[2]

z2


In [76]:
print z[2]()

3+v/((1+2)*ipar)


In [77]:
print z[0]

z2


In [78]:
print z[0]()

3+v/((1+0)*ipar)


In [79]:
print z[10]

IndexError: Index to multiple Quantity definition out of the valid range [0,5]

#More about the `QuantSpec` class

A `QuantSpec` object be considered a pre-`Quantity` form of raw symbolic expression, in that it has not yet been committed to a life as any particular Quantity sub-type (`Var`, `Par`, or `Fun`). It can be used in different ways in different `Quantity` definitions, and can be manipulated symbolically in its own right.

In [80]:
q = dst.QuantSpec('q', '1+k/2')

In [81]:
q

QuantSpec q (ExpFuncSpec)

In [82]:
q()

'1+k/2'

A major difference between a `Quantity` object and a `QuantSpec` object is that the `QuantSpec`'s name is not an important external part of its nature: only it's definition is important. Therefore, calling print on the object always returns its definition.

In [83]:
print q

1+k/2


In [84]:
print q()

1+k/2


In [85]:
print q.eval()

1+k/2


We can use a `QuantSpec` object to provide a definition for a `Quantity` object:

In [86]:
v = dst.Var(q, 'v')
v # this will print out the type of v

Var v (ExpFuncSpec)

In [87]:
v() # this will print out the type of v()

QuantSpec v (ExpFuncSpec)

In [88]:
(v())() # this will call the QuantSpec object returned by v()

'1+k/2'

In [89]:
w = dst.Var((q/q) + q, 'w') # as mentioned, can be manipulated symbolically

In [90]:
print w()

(1+k/2)/(1+k/2)+1+k/2


In [91]:
print 2 * q

2*(1+k/2)


For the most consistent results, any `QuantSpec` results that you want to use again should be defined into a proper `Var` or `Fun` object. Also, `QuantSpec`s are not a good place to manipulate vectors of symbols -- these should be converted to full Quantity types.

#Vectors as symbols

`Quantity` objects can define vectors of symbols (of up to rank 2). This can simplify notation, and permits `Fun` objects of several variables that can be symbolically differentiated. Examples of this are in `tests/Symbolic_Diff_test.py` and `tests/vode_withJac_Symbolic_test.py`.

In [92]:
x = dst.Var('x')
y = dst.Var('y')

f = dst.Var(['-3*Pow((2*x+1),3)+2*(x+y)', '-y/2'], 'f') # note that the definition is using a list!

print f[0] # only works if f is a Quantity type

-3*Pow((2*x+1),3)+2*(x+y)


In [93]:
f[0]

QuantSpec f_0 (ExpFuncSpec)

In [94]:
print f[1]

-y/2


Symbolic vectors can be defined for `QuantSpec`s and `Quantity`s, but are intended to be used in `Quantity` objects. This is because "array indexing" notation (using square brackets) is only properly supported for `Quantity`s. However, the `fromvector` method turns a vector `Quantity` or `QuantSpec` into a list of non-vector `QuantSpec` objects, each defining a component of the vector. It takes an optional integer index argument which selects that component of the vector only.

In [95]:
f.fromvector(0) # equivalent to f[1], but safe for both Quantities and QuantSpec



-3*Pow((2*x+1),3)+2*(x+y)


In [96]:
print f.fromvector(0)   

-3*Pow((2*x+1),3)+2*(x+y)


In [97]:
(f.fromvector(0))()

'-3*Pow((2*x+1),3)+2*(x+y)'

In [98]:
print f.fromvector()

[QuantSpec f_0 (ExpFuncSpec), QuantSpec f_1 (ExpFuncSpec)]


The method `tonumeric` reduces `Quantity`s whose definitions are entirely numeric to floating point numbers or arrays of such. 

In [99]:
df = dst.Diff(f, [x,y]) # differentiate each component of f with respect to x and y -- Jacobian, see next section
print df

[[-3*6*Pow((2*x+1),2)+2,2],[0,-0.5]]


In [100]:
dfe = df.eval(x=3,y=10)
print dfe

[[-880.0,2],[0,-0.5]]


In [101]:
dfe

QuantSpec __result__ (ExpFuncSpec)

In [102]:
dfe.tonumeric()

array([[ -8.80000000e+02,   2.00000000e+00],
       [  0.00000000e+00,  -5.00000000e-01]])

In [103]:
type(dfe.tonumeric())

numpy.ndarray

#Symbolic differentiation

Symbolic differentiation of mathematical expressions involving common mathematical functions is supported through the function `Diff`. 

The `Diff` function is meant to be a symbolic counterpart to the numerical derivative function `diff`, implemented in `common.py`, in the spirit of the Maple symbolic engine.

For many examples see the script `PyDSTool/tests/Symbolic_Diff_test.py`.

#Specification types

There are three specification sub-types of both Quantity and QuantSpec objects, determined by the 'specType' argument at initialization. These types are: 

* `'RHSfuncSpec'` (the named `Quantity` is defined as a right-hand side for differential or difference equation)
* `'ExpFuncSpec'` (the named `Quantity` is defined explicitly by the expression, such as for an auxiliary variable)
* `'ImpFuncSpec'` (the named `Quantity` is defined implicitly by the expression) 

The default is `'ExpFuncSpec'`. This type does not allow the `Quantity`'s name to appear in the defining expression, whereas the other two types do allow this.

In [104]:
f = dst.Var('2*k', 'f')

In [105]:
g = dst.Var('g(2*k)', 'g')

ValueError: Cannot define the symbol g in terms of itself with spec type ExpFuncSpec

In [106]:
g = dst.Var('g*2*k', 'g', specType="ImpFuncSpec")

In [107]:
h = dst.Var('h*2*k', 'h', specType="RHSfuncSpec")

These type strings are used by the `ModelConstructor` class to automatically determine how to treat each definition in a 'flattened' `ModelSpec` specification dictionary, and allows it to verify whether the types are compatible with the target `Generator` in which the specification will be instantiated. For instance, `'ImpFuncSpec'` definition types cannot be supplied to an `ExplicitFnGen` generator class

#Creation of Python functions from symbolic expressions

Symbolic expressions can be turned into Python functions using the `expr2fun` utility. The "free names" of an expression can either be numerically substituted from a supplied name -> value dictionary or left to become formal arguments to the function.

This utility can be handy to graph auxiliary functions from their abstract specifications (e.g., see `/examples/CIN.py`). Here is a simple example:

In [108]:
pyf = dst.expr2fun(f)
pyf

In [109]:
pyf(2)

4

In [110]:
vecpyf = np.vectorize(pyf)

In [111]:
vecpyf(np.arange(5))

array([0, 2, 4, 6, 8])

#Additional technical information

Both `Quantity` and `QuantSpec` objects have a `usedSymbols` attribute that shows which alphanumeric symbols were used in the object's definition (not including its name).

In [112]:
v.usedSymbols

['1', 'k', '2']

When defining a `Quantity`, additional information can be provided, such as the valid domain of the variable/parameter/function values. The domain defaults to the floating-point continuous domain `[-INF,INF]`.

#`Symbolic.py`

Preliminary efforts to document useful code inside `Symbolic.py` that hasn't been covered yet.

###`prepJacobian(varspecs, coords, fnspecs=None, max_iter_depth=20)`

Returns a symbolic Jacobian and updated function specs to support its definition from variable specifications. Only makes the Jacobian with respect to the named coordinates, which will be sorted into alphabetical order.

What is its purpose? Why does it not work in the examples below?

In [113]:
x = dst.Var('x')
y = dst.Var('y')

f = dst.Var(['-3*x', '-y/2'], 'f')
            
dst.prepJacobian(f, ['x', 'y'])

TypeError: list indices must be integers, not str

In [114]:
x = dst.Var('x')
y = dst.Var('y')

f = dst.Var(['-3*x', '-y/2'], 'f')
            
dst.prepJacobian(f, [x, y])

TypeError: list indices must be integers, not Var

In [115]:
dst.filteredDict(f, [0, 1])

{0: QuantSpec f_0 (ExpFuncSpec), 1: QuantSpec f_1 (ExpFuncSpec)}

In [116]:
dst.sortedDictValues(dst.filteredDict(f, [0, 1]))

[QuantSpec f_0 (ExpFuncSpec), QuantSpec f_1 (ExpFuncSpec)]

In [117]:
dst.Diff(dst.sortedDictValues(dst.filteredDict(f, [0, 1])), [0, 1])

TypeError: not all arguments converted during string formatting

###`QuantSpec.difference(other)`

In [118]:
x = dst.Var('x')
y = dst.Var('y')

f = dst.Var(['-3*x', '-y/2'], 'f')
g = dst.Var(['-2*x', '-y/2'], 'g')

f.difference(g)

Type:Var [True, False, True, True, True, False]


The output as it stands isn't human readable. How can we make it human readable? Maybe print out what the "key" associated with each True/False?

`results.append(type(self) == type(other))`

`results.append(self.subjectToken == other.subjectToken)`

`results.append(self.specStr.replace(" ","") == other.specStr.replace(" ",""))`

`results.append(self.specType == other.specType)`

`results.append(self.parser() == other.parser())`


In [119]:
f().subjectToken

'f'

In [120]:
g().subjectToken

'g'

In [121]:
f().parser()

'[-3*x,-y/2]'

###`Quantity.setDomain(domain)`

In [122]:
f.domain

(float, Continuous Domain, [-inf, inf])

In [123]:
f.setDomain([0, 100])
f.domain

(float, Continuous Domain, [0, 100])

`f` is a vector though, so what if I want to specify domains for each variable in `f`?

###`QuantSpec.isvector()`

In [124]:
f.isvector()

True

###`Quantity.redefine(specStr)`

In [125]:
f.redefine('ImpFuncSpec')

###`Quantity.isspecdefined()`

In [126]:
f.isspecdefined()

True

###`Quantity.renderForCode()`

Return code-ready version of spec (e.g. uncapitalize built-in function names).

In [127]:
f = dst.Var('Pow(k, 2)', 'f')
print f()

Pow(k,2)


In [128]:
g = f.renderForCode()
print g()

pow(k,2)


###`Quantity.rename('newName')`

In [129]:
print f

f


In [130]:
f.rename('fsharp')

In [131]:
print f

fsharp


###`Quantity.info()`

In [132]:
f.info()

Information for ModelSpec fsharp

dim: 0
domain: (Class float, Continuous Domain, [-inf, inf])
freeSymbols: [Pow, k]
funcSpecDict: < empty >
multiDefInfo: (False, , , 0, 0)
name: fsharp
spec: QuantSpec fsharp (ExpFuncSpec)
specType: ExpFuncSpec
usedSymbols: [Pow, k, 2]


#Functional specification using `FuncSpec`

The `FuncSpec.py` module deals with the specification of functionality of a Generator object. 

A `FuncSpec` object is passed to a `Generator` object when it is initialized and is used to build internal representations of its trajectory generation functions -- for instance, a function that computes the value of the right-hand side of an ODE, which can be passed to an ODE solver.

##Elements of a system specification

A system specification is made up of some or all of the following definitions:

* [required] name of model system 

* [required] state ("phase") variables and parameters

* auxiliary variables
    * Auxiliary variables are meant for the output of useful quantities derived from state variables and parameters. Thus, a specification of a state variable may not refer to an auxiliary variable. 
    
* auxliary ("helper") functions
    * If you want to avoid costly re-calculation of terms, either define auxiliary ("helper") functions or utilize the reuseterms option in your system definition. Auxiliary functions are also a way to create derived parameters.

* system parameters

* domains for state variables and parameters
    * Specification of valid domains for state variables, parameters, and the independent variable may be optional, depending on the target Generator. These are supplied as two-element lists containing the interval range endpoints that can include `Inf`. Also see [BoundsSafety](http://www.ni.gsu.edu/~rclewley/PyDSTool/BoundsSafety.html) for uses of domain information. 
    
* initial conditions for state variables

* autonomous external inputs

* user-defined zero-crossing events

* algorithmic parameters (e.g. integration tolerances)

* list of special names to ignore when parsing

* other generator-specific functions

These definitions are encapsulated in an `args` object so that they may be passed to a `Generator`:

In [137]:
example_args = dst.args(name='example_args', 
                        pars={'a': 3, 'b': 200},
                        varspecs={'x': 'y', 'y': 'myfunc(a, b)*x'},
                        fnspecs={'myfunc': (['u', 'v'], 'pi*u*v')})  # user-defined auxiliary function)

##Mathematical Expression syntax

Symbolic manipulation of the abstract syntax prior to "compilation" is done with the help of `Quantity` objects. The specification of structured models, that exhibit hierarchical modularity and other such symmetries, is accomplished using `ModelSpec` objects.

**Note:** There is a small amount of syntax checking provided at 'compile' time, mainly to check that all string tokens reference known variables, parameters, inputs, auxiliary functions or math objects. No semantic checking is done, and errors not caught at the time of initializing a Generator may lead to cryptic Python errors during trajectory generation.

Standard math names such as `sin`, `pow`, and `pi` do not need prefixing with a library reference such as in `math.sin` (for a Python target). The `sign` and `mod` functions are also available.

Time must be referenced as `t`, and references to variables, parameters, and external inputs is by the corresponding declared name. This is known as "index-free" notation. Internally, The `FuncSpec.py` module will translate these names to array index references once and for all. The user does not have to know or use these indices.

###Special functions

Presently, "special" functions (as defined by `scipy`) are only available in the Python-based integrators `Vode` and `Euler`. When using it, prefix the function name with `special_` in your definition string. The list of available functions is:

    airy, airye, ai_zeros, bi_zeros, ellipj, ellipk, ellipkinc, ellipe, ellipeinc, jn, jv, jve, yn, yv, yve, kn, kv, kve, iv, ive, hankel1, hankel1e, hankel2, hankel2e, lmbda, jnjnp_zeros, jnyn_zeros, jn_zeros, jnp_zeros, yn_zeros, ynp_zeros, y0_zeros, y1_zeros, y1p_zeros, j0, j1, y0, y1, i0, i0e, i1, i1e, k0, k0e, k1, k1e, itj0y0, it2j0y0, iti0k0, it2i0k0, besselpoly, jvp, yvp, kvp, ivp, h1vp, h2vp, sph_jn, sph_yn, sph_jnyn, sph_in, sph_kn, sph_inkn, riccati_jn, riccati_yn, struve, modstruve, itstruve0, it2struve0, itmodstruve0, bdtr, bdtrc, bdtri, btdtr, btdtri, fdtr, fdtrc, fdtri, gdtr, gdtrc, gdtria, nbdtr, nbdtrc, nbdtri, pdtr, pdtrc, pdtri, stdtr, stdtridf, stdtrit, chdtr, chdtrc, chdtri, ndtr, ndtri, smirnov, smirnovi, kolmogorov, kolmogi, tklmbda, gamma, gammaln, gammainc, gammaincinv, gammaincc, gammainccinv, beta, betaln, betainc, betaincinv, psi, digamma, rgamma, polygamma, erf, erfc, erfinv, erfcinv, erf_zeros, fresnel, fresnel_zeros, fresnelc_zeros, fresnels_zeros, modfresnelp, modfresnelm, lpn, lqn, lpmn, lqmn, lpmv, sph_harm, legendre, chebyt, chebyu, chebyc, chebys, jacobi, laguerre, genlaguerre, hermite, hermitenorm, gegenbauer, sh_legendre, sh_chebyt, sh_chebyu, sh_jacobi, hyp2f1, hyp1f1, hyperu, hyp0f1, hyp2f0, hyp1f2, hyp3f0, pbdv, pbvv, pbwa, pbdv_seq, pbvv_seq, pbdn_seq, mathieu_a, mathieu_b, mathieu_even_coef, mathieu_odd_coef, mathieu_cem, mathieu_sem, mathieu_modcem1, mathieu_modcem2, mathieu_modsem1, mathieu_modsem2, pro_ang1, pro_rad1, pro_rad2, obl_ang1, obl_rad1, obl_rad2, pro_cv, obl_cv, pro_cv_seq, obl_cv_seq, pro_ang1_cv, pro_rad1_cv, pro_rad2_cv, obl_ang1_cv, obl_rad1_cv, obl_rad2_cv, kelvin, kelvin_zeros, ber, bei, berp, beip, ker, kei, kerp, keip, ber_zeros, bei_zeros, berp_zeros, beip_zeros, ker_zeros, kei_zeros, kerp_zeros, keip_zeros, expn, exp1, expi, wofz, dawsn, shichi, sici, spence, zeta, zetac, cbrt, exp10, exp2, radian, cosdg, sindg, tandg, cotdg, log1p, expm1, cosm1, round

To use similar functions in C definitions you will have to include a C library defining them and call them with the imported names (see [here](http://www.ni.gsu.edu/~rclewley/PyDSTool/Generators.html#Making_changes_to_the_C_code_through_the_API)). You will have to let PyDSTool know which functions you are using so that it doesn't throw an error for finding an "undefined" name, using the `ignorespecial` keyword in the defining arguments.

##Macros

###Multiple state variable definitions using the `for` macro

A `for` loop macro is provided to ease repetitive definitions in all specification strings. This is pre-processed to 'unroll' the loop into a flat list of definitions before the parser processes the specification string. The syntax is:

    for(i, <ilo>, <ihi>, <expr_in_i>)
    
where `<ilo>` and `<ihi>` are integers, and the expression in `i` (or any other alphabet character) has all occurrences of `i` in square brackets replaced with the appropriate integer. Parameter declarations for such expressions are strings of the form `x[i]` for a variable `x`. Here is an example of usage:

In [138]:
for_demo = dst.args(name='for_demo',
                    pars={'p0': 0, 'p1': 1, 'p2': 2},
                    varspecs={'z[j]': 'for(j, 0, 1, 2*z[j+1] + p[j])', 'z2': '-z0 + p2'})   
                    # z2 is defined as the end case

###Summating using the `sum` macro

Similar to `for`, we have the `sum` macro:
    sum(i, <ilo>, <ihi>, <code_using_i>)

Here is an example:


In [139]:
sum_demo = dst.args(name='sum_demo',
                    pars={'p0': 0, 'p1': 1, 'p2': 2},
                    varspecs={'x': 'sum(i, 0, 4, sum(j, 0, 1, if([i]==2, 0, indexfunc([j] + p[j]))))'},
                    fnspecs={'indexfunc': (['x'], 'pi*x')}) # user-defined auxiliary function returning Pi * x
                    

##Auxiliary functions

Auxiliary functions are accessible to variable specifications and event definitions by name. As well as the option for user-defined functions, several in-built auxiliary functions are provided. Currently, these are:

* `if(<cond_expr>, <expr_1>, <expr_2>)` evaluates to `<expr_1>` if `<cond_expr>` evaluates to `True`, otherwise `<expr_2>` is evaluated.

* `heav(<expr>) = 1` if the `<expr>` evaluates to a positive value, otherwise = 0.

* `globalindepvar(<local_indepvar>)` returns the system's "global" independent variable (usually time) corresponding to the "local" value of the independent variable. See [HybridSystems](http://www.ni.gsu.edu/~rclewley/PyDSTool/HybridSystems.html).

* `initcond(<var_name>)` returns the initial condition of the named state variable.

The first two are present for dealing with discrete-time systems (i.e., MapSystem), or legacy ODE code (e.g., from XPP) when the discrete events they involve are not computed beyond the accuracy of the integrator's step size. For accurate discrete event handling in continuous-time systems, the hybrid system Model class should be used.

The user may define other auxiliary functions. These can help to simplify and organize the evaluation of right-hand sides, and to improve computational efficiency. User-defined functions are exposed in the generator as Python functions. For instance, a function `'h'` of one argument for a generator `g` is exposed as `g.auxfns.h` and has the same calling sequence as the internal function. Any parameter values used in the definition of the function are also mapped dynamically to the current values of `g.pars`.

Examples:

    def fsargs():
        fvarspecs = {
            'z[i]': 'for(i, 0, 1, 2*a+myauxfn1(2+t) - ' +
            'w*exp(-t)+ 3*z[i]**2 + z[i+1])',
            'z2': '2*a+myauxfn1(2+t) - w*exp(-t) + 3*z2**2 + z0',
            'w': 'k* w + a*sin(t)*itable + myauxfn1(t)*myauxfn2(w)/(a*sin(t))',
            'aux_wdouble': 'w*2 + globalindepvar(t)-0.5*sin(t)',
            'aux_other': 'myauxfn1(2*t) + initcond(w)',
            'aux_iftest': '1+if(z1>0,(1e-2+w), k+0.4)'
        }
        fnspecs = {
            'simpfn': ([''], '1+a'),
            'myauxfn1': (['t'], '2.5*cos(3*t)*simpfn()+sin(t)*2.e-3'),
            'myauxfn2': (['z0', 'w'], 'if(-z0<0,w/2+exp(-w),0)'),
            'myauxfn3': (['w'], 'k*w/2+ pow(w,-3.0+k)'),
            # not supposed to be the actual Jacobian!
            'Jacobian': (['t', 'z0', 'z1', 'z2', 'w'],
                         """[[2e5*z0+.1, -w/2, 0, 1.],
                         [-3.e-2 - a*w+z1, 1., z0+1, z0+sin(t)],
                         [k, w*z1+exp(-t), 0., z2/3+w],
                         [0, z1+a, 1., z0-w/3]]"""),
            # not supposed to be the actual Jacobian_pars!
            'Jacobian_pars': (['t', 'k', 'a'],
                              """[[0., 2.],
                              [z0, 1],
                              [3*w, 0.],
                              [2-z1, sin(t)*z2]]""")
        }
        return {
            'name': 'xfn',
            # vars is always unrolled by Gen base class if there are FOR loop macros
            'vars': ['w', 'z0', 'z1', 'z2'],
            'auxvars': ['aux_wdouble', 'aux_other', 'aux_iftest'],
            'pars': ['k', 'a'],
            'inputs': 'itable',
            'varspecs': fvarspecs,
            'fnspecs': fnspecs,
            'reuseterms': {'a*sin_t': 'ast',
                           'exp(-t)': 'expmt',
                           'sin(t)': 'sin_t',
                           'myauxfn1(2+t)': 'afn1call'},
            'codeinsert_end': """    print('code inserted at end')""",
            'codeinsert_start': """    print('code inserted at start')""",
            'targetlang': 'python'
        }


###`'reuseterms'`

    2*sin(afunc(p))+cos(afunc(p))*sin(afunc(p))-afunc(p)/5.

A reuseterms dictionary  `{'afunc(p)': 'afp', 'sin(afp)': 'sa', 'other(x)': 'ox'}`  will result in the final specification:

    2*sa+cos(afp)*sa-afp/5.

preceded by the declarations:

    double afp = afunc(p);
    double sa = sin(afp);
    (in C)

    afp = afunc(p)
    sa = sin(afp)
    (in Python)

#`ModelSpec` structured model specification

The goal of the `ModelSpec` classes is to provide a class hierarchy that allows complex dynamical models to be composed from simple units of mathematical expression. There is a low-level `QuantSpec` wrapper around a parser class, which in turn is built into the `Quantity` classes. These classes are really the elementary model building blocks, and have been the focus of the discussion up to this point.

Recall that we are able to define multiple quantities using indexed based notation (as discussed under **Multi-Reference Quantities**). 

At the next level of the model specification hierarchy are "components" (class `Component`). These may be either regular components or "leaf" components (class `LeafComponent`, a sub-class of `Component`) that cannot themselves contain other components, i.e. are at the "tips" of the tree structure.

What should `ModelSpec` classes abstract usually? Are there some heuristics to follow?

###`compatibleGens` and `targetLangs`

`PyDSTool` has a variety of generators that may be used in order to execute a given dynamical system:

In [3]:
dst.findGenSubClasses('Generator')

['Radau_ODEsystem',
 'ExplicitFnGen',
 'MapSystem',
 'ADMC_ODEsystem',
 'Vode_ODEsystem',
 'ImplicitFnGen',
 'LookupTable',
 'EmbeddedSysGen',
 'Euler_ODEsystem',
 'InterpolateTable',
 'Dopri_ODEsystem',
 'ExtrapolateTable']

Usually, the first step in defining a `ModelSpec` is deciding which type of `Generator`s our components will be compatible with. What is `targetLangs` for? This can be done as follows:

In [5]:
compatibleGenerators = dst.findGenSubClasses('ODEsystem') # compatible with ODESystem type generators

class ODEComponent(dst.Component):
    compatibleGens = compatibleGenerators
    targetLangs = dst.targetLangs  # (from common.py) -- all are compatible with ODEs
    
class ODELeafComponent(dst.LeafComponent):
    compatibleGens = compatibleGenerators
    targetLangs = dst.targetLangs  # (from common.py) -- all are compatible with ODEs

###`compatibleContainers` and `compatibleSubcomponents`

Once we have defined some components with compatibility constraints, we can begin to make use of them in order to define a model.

In [6]:
class Node(ODELeafComponent):
    pass

class Edge(ODELeafComponent):
    pass

class Lattice(ODEComponent):
    pass

For example, if we want to set up a system where `Nodes` and `Edges` are only contained by a `Lattice`, we would specify:

In [7]:
Lattice.compatibleSubcomponents=(Node,)
Node.compatibleContainers=(Lattice,)
Edge.compatibleContainers=(Lattice,)

###Functions that make use of components

In [136]:
def connectNodesWithEdge(edge_name, node1, node2):
    pass

An "edge" connects two nodes. For instance, if a node represents a mass, an edge could represent a spring. So, woudl `connectNodesWithEdge` in that case return the terms of the form $(k/m)x$? How would this eventually get parsed into a matrix? That is, how will the terms be given the right signs, and the right $m$ values, depending on which masses are being connected? 

Access down the levels of hierachical composition of model components is represented in the familiar "dot" notation of Python object-oriented classes. For instance, `mysuper.leaf1` and `mysuper.leaf2` would describe a Component `mysuper` that can contain LeafComponents `leaf1` and `leaf2`.

The `searchModelSpec` function is powerful way to access or probe the contents of a `ModelSpec` object according to a variety of search criteria. The criteria may include wild-cards, for instance, and makes use of the "dot" notation for representing hierarchical depth.

A range of information is held in a `_registry` attribute of a `ModelSpec` component. Sub-components can be added, renamed, and deleted.

#Transforming specifications into models

In order to turn a hierarchical specification of a model into a target `Generator` or `Model` object, the specification must be "flattened", and information about the sub-components that is embedded in this specification in a structured way must also be extracted and listed out. 

A `ModelSpec` component has a method `flattenSpec` which can be called once the model has been fully defined. A fully defined model is one that has no free names in need of resolving. The component has the attribute `freeSymbols`, but names external to the specification, such as `t` for time in non-autonomous systems, or names of built-in auxiliary functions, will be treated as free symbols. The `flattenSpec` function takes an optional list argument `globalRefs` to specify such names in order to complete the definition of the system.

##The `ModelConstructor` utility

The `ModelConstructor` classes simplify some of the tasks in converting a `ModelSpec` specification into a `FuncSpec` call that ultimately produces a target `Generator` or `Model` class instance (see page `FunctionalSpec`).

Models created in this way contain a copy of their source `ModelSpec` in the `mspecdict` attribute.

Examples of usage are provided in `/tests/CIN.py` and `/tests/ModelSpec_test.py`.